In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

In [2]:
%%yaml wmh_inference_config

log:
    root: /storage/vbutoi/scratch/ESE
    log_interval: 50

model:
    exp_name: 10_23_23_Dense_WMH
    num_workers: 1  

dataset:
    split: '?'
    task: '?'
    slicing: dense_full 
    input_type: volume

calibration:
    num_bins: 10
    bin_weightings:
        - proportional
        - uniform

<IPython.core.display.Javascript object>

In [3]:
%%yaml metric_cfg 

cal_metrics:
    ECE:
        func: ese.experiment.metrics.pixelwise.ECE
        color: blue
    ACE:
        func: ese.experiment.metrics.pixelwise.ACE
        color: goldenrod
    Island_ECE:
        func: ese.experiment.metrics.pixelwise.Island_ECE 
        color: lightgreen 
    ReCE:
        func: ese.experiment.metrics.regionwise.ReCE
        color: green

<IPython.core.display.Javascript object>

In [4]:
from ionpy.util import dict_product, Config

# Get the options for WMH
##################################################
wmh_dataset_options = {
    'dataset.split': ['train', 'val', 'cal'],
    'dataset.task': ['Singapore', 'Amsterdam']
}

base_wmh_cfg = Config(wmh_inference_config).update(metric_cfg)

wmh_cfgs = []
for cfg_update in dict_product(wmh_dataset_options):
    new_wmh_cfg = base_wmh_cfg.update(cfg_update)
    wmh_cfgs.append(new_wmh_cfg)

In [5]:
len(wmh_cfgs)

6

In [6]:
wmh_cfgs[0]

Config({'log': {'root': '/storage/vbutoi/scratch/ESE', 'log_interval': 50}, 'model': {'exp_name': '10_23_23_Dense_WMH', 'num_workers': 1}, 'dataset': {'split': 'train', 'task': 'Singapore', 'slicing': 'dense', 'input_type': 'volume'}, 'calibration': {'num_bins': 10, 'bin_weightings': ['proportional', 'uniform']}, 'cal_metrics': {'ECE': {'func': 'ese.experiment.metrics.pixelwise.ECE', 'color': 'blue'}, 'ACE': {'func': 'ese.experiment.metrics.pixelwise.ACE', 'color': 'goldenrod'}, 'Island_ECE': {'func': 'ese.experiment.metrics.pixelwise.Island_ECE', 'color': 'lightgreen'}, 'ReCE': {'func': 'ese.experiment.metrics.regionwise.ReCE', 'color': 'green'}}})

In [7]:
from ese.experiment.analysis.inference import get_cal_stats

get_cal_stats(wmh_cfgs[0])

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

ImportError: Could not import ese.experiment.metrics.pixelwise.ECE

In [ ]:
# from ionpy.slite.submit import submit_jobs
# from ese.experiment.analysis.inference import get_cal_stats

# submit_jobs(
#     project="ESE",
#     exp_name="10_09_23_WMH_and_BP_inference",
#     config_list=cfgs, 
#     job_func=get_cal_stats
#     )